In [1]:
import pandas as pd

In [150]:
import numpy as np

In [ ]:
class RecordedTrade(Item):
    ActionNum = Field()
    Time = Field()
    Type = Field()
    Order = Field()
    Size = Field()
    Price = Field()
    SL = Field()
    TP = Field()
    Profit = Field()
    Balance = Field()

class TradeSpider(BaseSpider):
    "Spider for extracting "

Objectives:
    * scrape trade data off the htm files put out as report by mT4
    * get candle data exported from the history center as a csv file
    * do some analytics that include
    * get trade setup
        * calculate min. win rate at trade entry, trade modification (if any) and trade exit
    * get corresponding candle data
        * compare body/whisker length at trade entry, trade modification (if any) and trade exit
        * track trade volume throughout
    * find whether there are any links between these data and trade outcomes

In [6]:
candleFname = 'EURUSD60.csv'
candleDF = pd.read_csv(candleFname,names=['DATE','TIME','OPEN','HIGH','LOW','CLOSE','VOLUME'],index_col=['DATE','TIME'])

In [12]:
candleDF.head()

,,OPEN,HIGH,LOW,CLOSE,VOLUME
DATE,TIME,,,,,
1971.01.03,18:00,0.5369,0.5369,0.5369,0.5369,1
1971.01.04,18:00,0.5366,0.5366,0.5366,0.5366,1
1971.01.05,18:00,0.5365,0.5365,0.5365,0.5365,1
1971.01.06,18:00,0.5368,0.5368,0.5368,0.5368,1
1971.01.07,18:00,0.5371,0.5371,0.5371,0.5371,1


In [22]:
candleDF.loc['2014.12.01','05:00']

OPEN         1.24415
HIGH         1.24689
LOW          1.24393
CLOSE        1.24644
VOLUME    3663.00000
Name: (2014.12.01, 05:00), dtype: float64

In [23]:
candleDF.loc['2014.12.01','05:00'].OPEN

1.2441500000000001

In [22]:
from bs4 import BeautifulSoup
import urllib2
import pandas as pd 
from collections import OrderedDict
url='file:///Users/grayWolf/Work/FX/StrategyTester.htm'
#def ParseReport(url):
page = urllib2.urlopen(url)
soup = BeautifulSoup(page,'html.parser')

all_tables = soup.find_all('table')

# get initial deposit
summaryTable=all_tables[0]
rows = summaryTable.findAll("tr")
initial_deposit=0
for row in rows:
    cell = row.findAll("td")
    if "Initial deposit" in cell[0]:
        initial_deposit = str(cell[1].find(text=True))
        break

# fill trade history dictionary
tradeTable=all_tables[-1]
rows=tradeTable.findAll("tr")

tradeHistTags = ["Action Number","Order Time","Action","Order Number",
                 "Order Vol.","Open Price","S/L","T/P","Profit","Balance"]

#tradeHist= [[] for i in range(10)]
tradeHist = OrderedDict.fromkeys(tradeHistTags)
for t in tradeHistTags:
    tradeHist[t] = []
#tradeHist={k:[] for k in tradeHistTags}
record = False
for row in rows:
    cell = row.findAll("td")
    if record == False:
        if "#" in cell[0].find(text="#"):
            record=True
            continue
    else:
        
        for i,entry in  enumerate(cell):
            
            result = (entry.find(text=True))
            tradeHist[tradeHistTags[i]].append(result)
            #tradeHist[i].append(result)
            
        if len(cell)<10:
            tradeHist[tradeHistTags[9]].append("None")
tradeHist["Balance"][0] = initial_deposit       

In [23]:
len(rows)

574

In [24]:
for k in tradeHist.keys():
    print k,len(tradeHist[k])

Action Number 573
Order Time 573
Action 573
Order Number 573
Order Vol. 573
Open Price 573
S/L 573
T/P 573
Profit 573
Balance 573


In [25]:
df=pd.DataFrame.from_dict(tradeHist)

In [26]:
df.set_index(['Action Number'])

,Order Time,Action,Order Number,Order Vol.,Open Price,S/L,T/P,Profit,Balance
Action Number,,,,,,,,,
1,2014.01.02 02:42,sell,1,1.00,1.37367,0.00000,0.00000,None,10000.00
2,2014.01.02 02:42,modify,1,1.00,1.37367,1.38067,1.37208,None,None
3,2014.01.02 03:13,t/p,1,1.00,1.37208,1.38067,1.37208,159.00,10159.00
4,2014.01.03 04:10,sell,2,1.00,1.36292,0.00000,0.00000,None,None
5,2014.01.03 04:10,modify,2,1.00,1.36292,1.36722,1.35991,None,None
6,2014.01.03 12:05,t/p,2,1.00,1.35991,1.36722,1.35991,301.00,10460.00
7,2014.01.05 18:00,sell,3,1.00,1.35955,0.00000,0.00000,None,None
8,2014.01.05 18:00,modify,3,1.00,1.35955,1.36584,1.35945,None,None
9,2014.01.05 18:01,t/p,3,1.00,1.35945,1.36584,1.35945,10.00,10470.00


In [28]:
df.to_excel('test.xlsx', sheet_name='Sheet1',index=False)